<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_RecSys/blob/Paul-Himyak-folder/Paul%20Himyak/save_modified_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!wget -qc "https://storage.yandexcloud.net/terratraineeship/23_InspectrumClinic_RS/datasets/%D0%B4%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82%20%D0%BF%D1%80%D0%BE%D1%84%D0%BF%D0%B0%D1%82%D0%BE%D0%BB%D0%BE%D0%B3%20%D1%87%D0%B0%D1%81%D1%82%D1%8C.xlsx"


In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import uuid
import re
import numpy as np
import seaborn as sns
from datetime import datetime

In [7]:
from google.colab import drive

# Монтируем Google Диск
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_excel("/content/датасет профпатолог часть.xlsx")


In [8]:
FIELD_CLIENT_AGE = "КлиентВозраст"
FIELD_CLIENT_BIRTH_DAY = "КлиентДатаРождения"
FIELD_CLIENT_GENDER = "КлиентПол"
FIELD_CLIENT_GENDER_M = 0
FIELD_CLIENT_GENDER_F = 1
FIELD_CONCLUSION = "ЗаключениеМК"
FIELD_CONCLUSION_FIT = 0
FIELD_CONCLUSION_PART_FIT = 1
FIELD_CONCLUSION_NOT_FIT = 2

FIELD_DISPANSARY_GROUP = "ДиспансернаяГруппа"
FIELD_DISPANSARY_GROUP_I = 0
FIELD_DISPANSARY_GROUP_II = 1
FIELD_DISPANSARY_GROUP_III = 2
FIELD_DISPANSARY_GROUP_OTHER = 3

FIELD_HARMFUL_FACTORS = "ВредныеФакторы"

FIELD_CLIENT_PROFESSION = "Профессия"

In [9]:
mkb_columns = df.filter(regex="(?i)МКБ10").columns

In [13]:
additionals_columns = df.filter(regex="(?i)Дополнение_Значение$").columns


In [14]:
base_columns = [FIELD_CLIENT_BIRTH_DAY, FIELD_CLIENT_GENDER, FIELD_CONCLUSION, FIELD_DISPANSARY_GROUP, FIELD_HARMFUL_FACTORS, FIELD_CLIENT_PROFESSION]

In [15]:
general_columns = np.concatenate((base_columns, mkb_columns, additionals_columns))

In [16]:
general_columns

array(['КлиентДатаРождения', 'КлиентПол', 'ЗаключениеМК',
       'ДиспансернаяГруппа', 'ВредныеФакторы', 'Профессия',
       'ПсихиатрияНаркология1_МКБ101', 'Офтальмология1_МКБ101',
       'Оториноларингология1_МКБ101', 'ХирургНеИсп1_МКБ101',
       'Гинекология1_МКБ101', 'Стоматология1_МКБ101',
       'Дерматовенерология1_МКБ101', 'Рентгенология1_МКБ101',
       'Неврология1_МКБ101', 'Терапия1_МКБ101', 'Хирургия1_МКБ101',
       'Офтальмология1_МКБ102', 'Терапия2_МКБ101', 'Неврология2_МКБ101',
       'ЛабораторныеИсследования1_МКБ101', 'ЭКГНеИсп1_МКБ101',
       'УЗИДиагностика1_МКБ101', 'УЗИДиагностика2_МКБ101',
       'ПсихиатрияНаркология2_МКБ101', 'Терапия1_МКБ102',
       'ЛабораторныеИсследования2_МКБ101', 'Терапия1_МКБ103',
       'Психиатрия1_МКБ101', 'Гинекология1_МКБ102',
       'Рентгенология2_МКБ101', 'Гинекология2_МКБ101', 'Хирургия2_МКБ101',
       'ПроцедурнаяМедсестра1_МКБ101',
       'ФункциональнаяДиагностика1_МКБ101', 'УЗИДиагностика1_МКБ102',
       'Стоматология2_

In [17]:
df[general_columns].to_excel("/content/drive/MyDrive/profpatolog_modified.xlsx")